# Implementation of the LNMMSB model (static)

In [ ]:

import jax
import jax.numpy as jnp
import jax.scipy as jsp
from jax import vmap, jit
from jax.tree_util import register_pytree_node_class
from functools import partial


In [ ]:
class LNMMSB():
    def __init__(self, nodes, roles, key=None):
        self.N = nodes
        self.K = roles

        if key is None:
            key = jax.random.PRNGKey(0)
        else:
            key = jax.random.PRNGKey(key)

        # Initialize model parameters
        self.B = jax.random.uniform(key, (self.K, self.K)) # shape (K,K)
        self.mu = jax.random.multivariate_normal(key, jnp.zeros(self.K), jnp.eye(self.K)) # shape (K,)
        self.Sigma = jnp.eye(self.K) * 10 #shape (K,K)

        self.gamma_tilde = None
        self.Sigma_tilde = None

        EPS = 1e-8

    def _compute_deltas(self, gamma_tilde, E):
        '''
        Compute delta matrices for all pairs (i,j) given current gamma_tilde and B
        gamma_tilde: (N,K)
        E: adjacency matrix (N,N)
        Returns: delta (N,N,K,K)
        '''
        gamma_i = gamma_tilde[:, None, :, None] # shape (N,1,K,1)
        gamma_j = gamma_tilde[None, :, None, :] # shape (1,N,1,K)

        gamma_sum = gamma_i + gamma_j # shape (N,N,K,K)

        B_reshaped = self.B[None, None, :, :] # shape (1,1,K,K)
        E_reshaped = E[:, :, None, None] # shape (N,N,1,1)

        bernoulli_term = jnp.where(E_reshaped == 1, B_reshaped, 1 - B_reshaped) # shape (N,N,K,K)

        delta = jnp.exp(gamma_sum) * bernoulli_term # shape (N,N,K,K)
        delta = delta / jnp.sum(delta, axis=(-2,-1), keepdims=True)# + EPS) # normalize over (K,K)

        return delta # shape (N,N,K,K)

    def log_likelihood(self, E):
        '''
        Compute log likelihood of data E given current parameters and deltas. Based on eqn (23).
        E: adjacency matrix (N,N)
        Returns: scalar log likelihood
        '''
        delta = self._compute_deltas(E) # shape (N,N,K,K)
        E_reshaped = E[:, :, None, None] # shape (N,N,1,1)
        B_reshaped = self.B[None, None, :, :] # shape (1,1,K,K)
        
        logB = jnp.log(B_reshaped + EPS) # shape (1,1,K,K)
        log1mB = jnp.log(1 - B_reshaped + EPS) # shape (1,1,K,K)
 
        ll_matrix = E_reshaped * logB + (1 - E_reshaped) * log1mB # shape (N,N,K,K)
        ll = jnp.sum(delta * ll_matrix) # scalar

        return ll
    
    def compute_g_H(self, gamma_hat):
        '''
        Compute g and H at gamma_hat
        gamma_hat: (N, K)
        Returns: g: (N, K), H: (N, K, K)
        '''
        g = jnp.exp(gamma_hat) / jnp.sum(jnp.exp(gamma_hat), axis=-1, keepdims=True) # shape (N,K)
        H = jnp.einsum('ni,nj->nij', g, jnp.eye(self.K)) - jnp.einsum('ni,nj->nij', g, g) # shape (N,K,K)
        return g, H
    
    def update_sigma_tilde(self, Sigma_inv, H):
        '''
        Compute Sigma_tilde = (Sigma^{-1} + (2N-2) H)^{-1}
        Sigma_inv: (K,K)
        H: (N,K,K) Hessian at gamma_hat
        Returns: Sigma_tilde: (N,K,K)
        '''
        factor = 2.0 * self.N - 2.0
        A = Sigma_inv[None, :, :] + factor * H # shape (N,K,K)
        #jitter = 1e-6 * jnp.eye(self.K)
        #A = A + jitter[None, :, :]
        Sigma_tilde = jnp.linalg.inv(A)
        return Sigma_tilde # shape (N,K,K)

    def update_gamma_tilde(self, mu, Sigma_tilde, gamma_hat, g , H):
        '''
        Update gamma_tilde using Laplace approximation

        mu: (K,)
        Sigma_tilde: (N,K,K)
        gamma_hat: (N,K)
        m_expect: (N,K)
        Returns: gamma_tilde: (N,K), Sigma_tilde: (N,K,K)
        '''
        # g, H = self.compute_g_H(gamma_hat) # g: (N,K), H: (N,K,K)
        factor = 2.0 * self.N - 2.0 #scalar

        m_expect = self.compute_m_expect(self.delta) # shape (N,K)

        term_1 = m_expect - factor * g + factor * jnp.einsum('nij,nj->ni', H, gamma_hat) - factor * jnp.einsum('nij,j->ni', H, mu) # shape (N,K)

        gamma_tilde = mu[None, :] + jnp.einsum('nij,nj->ni', Sigma_tilde, term_1) # shape (N,K)

        return gamma_tilde # shape (N,K)

    def compute_m_expect(self, delta):
        '''
        Compute m_expect per node: m_i,k = sum_{j != i} (E[z_i->j,k] + E[z_i<-j,k])
        delta: (N,N,K,K)
        Returns: m_expect: (N,K)
        '''
        z_ij = jnp.sum(delta, axis=-1) # shape (N,N,K) Expected z_i->j
        z_ji = jnp.sum(delta, axis=-2) # shape (N,N,K) Expected z_j<-i

        z_sum = z_ij + z_ji # shape (N,N,K)

        m_expect = jnp.sum(z_sum, axis=1) - jnp.diagonal(z_sum, axis1=0, axis2=1) # shape (N,K)

        return m_expect # shape (N,K)
    
    def update_B(self, E, delta):
        '''
        Update B using eq (24): β_{k,l} = sum_{i,j} eij δij(k,l) / sum_{i,j} δij(k,l)
        E: adjacency matrix (N,N)
        delta: (N,N,K,K)
        Returns: updated B (K,K)
        '''
        extended_E = E[:, :, None, None] # shape (N,N,1,1)

        num = jnp.sum(extended_E * delta, axis=(0,1)) # shape (K,K)
        den = jnp.sum(delta, axis=(0,1)) # shape (K,K)

        B_new = (num + EPS) / (den + EPS) # shape (K,K)
        B_new = jnp.clip(B_new, 1e-6, 1 - 1e-6)

        return B_new # shape (K,K)
    
    def update_mu_sigma(self, gamma_tilde, Sigma_tilde):
        '''
        Update mu and Sigma based on eq (13)
        gamma_tilde: (N,K)
        Sigma_tilde: (N,K,K)
        Returns: updated mu (K,), updated Sigma (K,K)
        '''
        mu = jnp.mean(gamma_tilde, axis=0) # shape (K,)

        avg_sigma_tilde = jnp.mean(Sigma_tilde, axis=0)  # shape (K,K)
        cov_gamma_tilde = jnp.cov(gamma_tilde, rowvar=False) # shape (K,K)

        updated_Sigma = avg_sigma_tilde + cov_gamma_tilde # shape (K,K)
        Sigma = updated_Sigma

        return mu, Sigma

    
    def fit(self, E, max_inner_iters=100, max_outer_iters=100, tol=1e-6, verbose=False):
        '''
        Fit the model to adjacency matrix E using variational EM
        Algorithm described in section 4.2 of the paper
        
        E: adjacency matrix (N,N)
        max_inner_iters: maximum iterations for inner loop
        max_outer_iters: maximum iterations for outer loop
        tol: tolerance for convergence
        verbose: whether to print progress
        '''
        
        i = 0 
        d_ll = jnp.inf
        prev_outer_ll = -jnp.inf
        while(d_ll > tol and i < max_outer_iters): # 2 (outer loop)
            if verbose:
                print(f"[outer {i}] mu: {self.mu}, Sigma diag: {jnp.diag(self.Sigma)}, B: {self.B}")

            #initialize q(gamma) parameters
            self.gamma_tilde = jax.random.multivariate_normal(jax.random.PRNGKey(outer), self.mu, self.Sigma, (self.N,)) # shape (N,K)
            g, H = self.compute_g_H(self.gamma_tilde) # g: (N,K), H: (N,K,K)
            Sigma_inv = jnp.linalg.inv(self.Sigma) # shape (K,K
            self.Sigma_tilde = self.update_sigma_tilde(Sigma_inv, H) # shape (N,K,K)

            #NOTE:add multiple runs with different initializaitons and use of VMAP
            j = 0
            inner_d_ll = jnp.inf
            prev_inner_ll = -jnp.inf
            while(d_ll > tol and j < max_inner_iters): # 2.2 inner loop
                # 2.2.1 update q_z (delta)
                self.delta = self._compute_deltas(self.gamma_tilde, E) # shape (N,N,K,K)

                # 2.2.2 update q_gamma parameters
                gamma_hat = self.gamma_tilde # shape (N,K)
                g, H = self.compute_g_H(gamma_hat) # g: (N,K), H: (N,K,K)
                self.Sigma_tilde = self.update_sigma_tilde(Sigma_inv, H) # shape (N,K,K)
                self.gamma_tilde = self.update_gamma_tilde(self.mu, self.Sigma_tilde, gamma_hat, g , H) # shape (N,K)

                # 2.2.3 update B
                self.B = self.update_B(E, self.delta) # shape (K,K)

                #convergence check
                j += 1
                inner_ll = self.log_likelihood(E) 
                d_ll = jnp.abs(inner_ll - prev_inner_ll)
                prev_inner_ll = inner_ll

                if verbose:
                    print(f"  [inner {j}] ll: {inner_ll:.4f}, d_ll: {inner_d_ll:.6f}")

            # 2.3 update mu, Sigma
            self.mu, self.Sigma = self.update_mu_sigma(self.gamma_tilde, self.Sigma_tilde) # mu: (K,), Sigma: (K,K)

            #convergence check
            i += 1
            outer_ll = inner_ll #last inner ll is outer ll
            d_ll = jnp.abs(outer_ll - prev_outer_ll)
            prev_outer_ll = outer_ll






                



## Test

In [25]:
model = LNMMSB(10, 3)

#### Sanity check

In [26]:
import numpy as np
gamma_tilde = jnp.arange(16).reshape(4,4).astype(jnp.float32)

print("gamma tilde:", gamma_tilde)
gamma_i = gamma_tilde[:, None, :, None]
gamma_j = gamma_tilde[None, :, None, :]


gamma_sum = gamma_i + gamma_j

print(gamma_sum.shape)
# print(gamma_sum)

N = 4
K = 4
delta = np.zeros((4, 4, 4, 4))
gamma_means = gamma_tilde

for i in range(N):
    for j in range(N):
        for u in range(K):
            for v in range(K):
                delta[i, j, u, v] = gamma_means[i, u] + gamma_means[j, v]

print(jnp.all(delta == gamma_sum))

        

gamma tilde: [[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
(4, 4, 4, 4)
True
